In [4]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [11]:
import pandas as pd

# Load the dataset
file_path = '/content/gdrive/MyDrive/Data Mining Project/Pharmacy Sales Clean.xlsx'
df = pd.read_excel(file_path)

# Display the first few rows of the dataset
df.head()


,Invoice Number,Date,Quantity,Amount,Item Code,Item Description,Branch Code,Shop Name,Customer Name,Sales Person Code,Sales Person Name,Cashier Name,Cashier Code,Sales Type,Unit Price
0,CS-04-068639/21,"Friday, January 1, 2021",1.0,10.0,27223,Face mask Disposable 3PLY Of 50 Pcs,PST-04,Axum no 4 pharmacy,Walkin Customer,18,Asefa Degefaw,Hirut Asefa,CS0008,Cash Sales,10.0
1,CS-04-068613/21,"Friday, January 1, 2021",1.0,10.0,27223,Face mask Disposable 3PLY Of 50 Pcs,PST-04,Axum no 4 pharmacy,Walkin Customer,18,Asefa Degefaw,Hirut Asefa,CS0008,Cash Sales,10.0
2,CS-04-068641/21,"Friday, January 1, 2021",1.0,10.0,27223,Face mask Disposable 3PLY Of 50 Pcs,PST-04,Axum no 4 pharmacy,Walkin Customer,18,Asefa Degefaw,Hirut Asefa,CS0008,Cash Sales,10.0
3,CS-04-068632/21,"Friday, January 1, 2021",1.0,10.0,27223,Face mask Disposable 3PLY Of 50 Pcs,PST-04,Axum no 4 pharmacy,Walkin Customer,18,Asefa Degefaw,Hirut Asefa,CS0008,Cash Sales,10.0
4,CS-04-068638/21,"Friday, January 1, 2021",1.0,10.0,27223,Face mask Disposable 3PLY Of 50 Pcs,PST-04,Axum no 4 pharmacy,Walkin Customer,18,Asefa Degefaw,Hirut Asefa,CS0008,Cash Sales,10.0


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347510 entries, 0 to 347509
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Invoice Number     347510 non-null  object 
 1   Date               347510 non-null  object 
 2   Quantity           347510 non-null  float64
 3   Amount             347510 non-null  float64
 4   Item Code          347510 non-null  int64  
 5   Item Description   347510 non-null  object 
 6   Branch Code        347510 non-null  object 
 7   Shop Name          347510 non-null  object 
 8   Customer Name      347509 non-null  object 
 9   Sales Person Code  347510 non-null  int64  
 10  Sales Person Name  347510 non-null  object 
 11  Cashier Name       347510 non-null  object 
 12  Cashier Code       347510 non-null  object 
 13  Sales Type         347510 non-null  object 
 14  Unit Price         347510 non-null  float64
dtypes: float64(3), int64(2), object(10)
memory usage: 3

In [13]:
# Display summary statistics for numerical columns
summary_statistics = df.describe()

# Display the DataFrame
from IPython.display import display
display(summary_statistics)

,Quantity,Amount,Item Code,Sales Person Code,Unit Price
count,347510.000000,347510.000000,347510.000000,347510.000000,347510.000000
mean,2.064868,209.145841,22096.743420,74.091724,129.924348
std,57.625409,778.778312,6955.195905,77.145282,265.879736
min,0.001000,-10391.500000,11001.000000,1.000000,0.500000
25%,1.000000,27.000000,17223.000000,18.000000,21.000000
50%,1.000000,56.000000,22060.000000,30.000000,46.960000
75%,1.500000,172.000000,27223.000000,110.000000,125.000000
max,31200.000000,149842.000000,39044.000000,261.000000,16228.000000


In [17]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder


# Inspect the date column
print("Date column sample:", df['Date'].head())

# Convert 'Invoice Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# One-hot encode the categorical feature 'Branch Code'
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_data = encoder.fit_transform(df[['Branch Code']])
encoded_df = pd.DataFrame(encoded_data.toarray(), columns=encoder.get_feature_names_out(['Branch Code']))

# Extract numerical features
numerical_features = ['Amount', 'Unit Price']

# Extract date features
date_feature = 'Date'
df['month'] = df[date_feature].dt.month
df['year'] = df[date_feature].dt.year
df['day_of_week'] = df[date_feature].dt.day_name()

# Combine all features
features = numerical_features + list(encoded_df.columns) + ['month', 'year', 'day_of_week']
df = pd.concat([df, encoded_df], axis=1)

# Convert 'day_of_week' to numerical format
df['day_of_week'] = df['day_of_week'].astype('category').cat.codes

# Prepare the feature matrix
X = df[features].values

# Determine the number of clusters (you can use techniques like elbow method or silhouette analysis)
n_clusters = 10  # Assuming you want 10 clusters

# Perform clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans.fit(X)
labels = kmeans.labels_

# Create a new DataFrame with the cluster labels and the original data
cluster_data = pd.DataFrame(data=X, columns=features)
cluster_data['cluster'] = labels

# Sample a fixed fraction of data points from each cluster
sample_fraction = 10000 / len(df)  # Adjusted to reduce the dataset to less than 10,000 rows

# Ensure the sample fraction does not exceed 1
if sample_fraction > 1:
    sample_fraction = 1

print(f"Sample fraction: {sample_fraction}")

sampled_data = cluster_data.groupby('cluster').apply(lambda x: x.sample(frac=sample_fraction))

# Reset the index and drop the cluster column
sampled_data = sampled_data.reset_index(drop=True)
sampled_data = sampled_data.drop('cluster', axis=1)

# Verify the reduced dataset
print(f"Original dataset shape: {df.shape}")
print(f"Sampled dataset shape: {sampled_data.shape}")

# You can continue working with the 'sampled_data' DataFrame
# ... your code for algorithms, models, etc. ...


Date column sample: 0   2021-01-01
1   2021-01-01
2   2021-01-01
3   2021-01-01
4   2021-01-01
Name: Date, dtype: datetime64[ns]


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Sample fraction: 0.028776150326609306
Original dataset shape: (347510, 30)
Sampled dataset shape: (10000, 17)


In [18]:
# Export the reduced dataset to an Excel file
output_file_path = '/content/gdrive/MyDrive/Data Mining Project/reduced_dataset.xlsx'
sampled_data.to_excel(output_file_path, index=False)

print(f"Reduced dataset exported to: {output_file_path}")

Reduced dataset exported to: /content/gdrive/MyDrive/Data Mining Project/reduced_dataset.xlsx
